# Klook Offer Curation

This notebook loads raw Klook offers, structures the relevant content, and uses the OpenAI Responses API to produce curation scores

In [31]:
import json
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path
from typing import Any, Dict, Iterable, List
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

import pandas as pd
from openai import OpenAI
from IPython.display import display

# Ensure DataFrame columns like 'reason' show full content
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

## Data Import and Structuring
Helpers for loading raw offer JSON files and shaping them into the consistent structure used downstream (`load_offers`, `extract_images`, `render_sections`, `structure_activity`).


In [35]:

OFFERS_DIR = Path("offers")


def load_offers(directory: Path) -> List[Dict[str, Any]]:
    """Load all JSON offers in the given directory."""
    offers: List[Dict[str, Any]] = []
    for path in sorted(directory.glob("*.json")):
        with path.open("r", encoding="utf-8") as fh:
            payload = json.load(fh)
        activity = payload.get("activity")
        if activity:
            offers.append({"path": str(path), "activity": activity})
    return offers


def extract_image_details(images: Iterable[Dict[str, Any]] | None) -> List[Dict[str, Any]]:
    """Collect image metadata with unique URLs preserved."""
    details: List[Dict[str, Any]] = []
    if not images:
        return details
    for image in images:
        if not isinstance(image, dict):
            continue
        url = image.get("image_url_host")
        if url:
            details.append(
                {
                    "url": url,
                    "type": image.get("image_type"),
                    "alt": image.get("image_alt"),
                    "description": image.get("image_desc"),
                    "width": image.get("width"),
                    "height": image.get("height"),
                    "source": "primary",
                }
            )
        nested = image.get("images")
        if isinstance(nested, list):
            for nested_image in nested:
                if not isinstance(nested_image, dict):
                    continue
                nested_url = nested_image.get("image_url_host")
                if not nested_url:
                    continue
                details.append(
                    {
                        "url": nested_url,
                        "type": nested_image.get("image_type"),
                        "alt": nested_image.get("image_alt"),
                        "description": nested_image.get("image_desc"),
                        "width": nested_image.get("width"),
                        "height": nested_image.get("height"),
                        "source": "nested",
                    }
                )
    seen: set[str] = set()
    unique_details: List[Dict[str, Any]] = []
    for detail in details:
        url = detail.get("url")
        if not url or url in seen:
            continue
        seen.add(url)
        unique_details.append(detail)
    return unique_details


def extract_images(images: Iterable[Dict[str, Any]] | None) -> List[str]:
    """Flatten the image list into absolute URLs."""
    return [detail["url"] for detail in extract_image_details(images)]


def render_sections(section_info: Iterable[Dict[str, Any]] | None) -> str:
    """Convert section metadata into markdown text."""
    if not section_info:
        return ""
    chunks: List[str] = []
    for section in section_info:
        if not isinstance(section, dict):
            continue
        section_name = (section.get("section_name") or "").strip()
        group_blocks: List[str] = []
        for group in section.get("groups", []):
            if not isinstance(group, dict):
                continue
            group_name = (group.get("group_name") or "").strip()
            content = (group.get("content") or "").strip()
            if group_name and content:
                group_blocks.append(f"### {group_name}\n{content}")
            elif content:
                group_blocks.append(content)
        body = "\n\n".join([block for block in group_blocks if block])
        if section_name and body:
            chunks.append(f"## {section_name}\n{body}")
        elif body:
            chunks.append(body)
    return "\n\n".join([chunk for chunk in chunks if chunk])


def structure_activity(activity: Dict[str, Any], source_path: str) -> Dict[str, Any]:
    """Extract the fields needed for grading from an activity payload."""
    packages: List[Dict[str, Any]] = []
    for package in activity.get("package_list", []) or []:
        if not isinstance(package, dict):
            continue
        packages.append(
            {
                "package_id": package.get("package_id"),
                "package_name": package.get("package_name"),
                "sections_markdown": render_sections(package.get("section_info")),
            }
        )
    city_info = activity.get("city_info") or []
    primary_city = city_info[0] if city_info else {}
    category_info = activity.get("category_info") or {}
    status = activity.get("status") or activity.get("curation_status") or category_info.get("curation_status")
    image_details = extract_image_details(activity.get("images"))

    return {
        "source_path": source_path,
        "activity_id": activity.get("activity_id"),
        "title": activity.get("title"),
        "subtitle": activity.get("subtitle"),
        "what_we_love": activity.get("what_we_love"),
        "location": activity.get("location"),
        "address": activity.get("address_desc_multilang"),
        "category": category_info.get("sub_category_name"),
        "category_detail": category_info,
        "description_markdown": render_sections(activity.get("section_info")),
        "packages": packages,
        "images": [item["url"] for item in image_details],
        "image_details": image_details,
        "city": primary_city.get("city_name"),
        "country": primary_city.get("country_name"),
        "status": status,
        "raw": activity,
    }


## Curation via GPT
Utilities that prepare prompts, call the OpenAI Responses API, and interpret grading results (`load_api_key_from_file`, `load_api_key`, `get_client`, `summarise_packages`, `build_offer_prompt`, `collect_response_text`, `parse_json_response`, `grade_offer`, `grade_offers_parallel`).


In [44]:

SYSTEM_PROMPT = """You are a senior Luxury Escapes curation editor. Evaluate each Klook offer for suitability on our platform.
Consider title clarity, image relevance, hero image suitability, category accuracy, description quality, and location correctness.
Return a strict JSON object with keys:
- score (0-5, integer)
- categories (array of categories that best describe the Klook activity. You can only choose from the list below)
- target_audiences (array of target audiences that best describe the Klook activity. You can only choose from the list below)
- hero_image_index (integer index from the numbered image list, starting at 1, or null if no supplied image is suitable)
- hero_image_url (string URL of the hero image that matches the numbered list entry, or null if none are acceptable)
- hero_image_reason (string explaining why the selected image works, or why none are acceptable)
- reason (concise justification including any category recommendations or red flags).

## Categories
These are the possible categories, note each is nested in a parent category. Do not include the parent category in the array.

{
  "Wine & Dine": [
    "Fine dining",
    "Restaurants & bars",
    "Cafés",
    "High tea",
    "Food tours",
    "Wine country trips",
    "Breweries, distilleries & vineyards"
  ],
  "Top Activities": [
    "Yachts, boats & cruises",
    "Cooking classes",
    "Up in the air",
    "Outdoor activities",
    "Watersports",
    "Indoor activities",
    "Photoshoot - Travelshoot",
    "Wildlife Cruises",
    "Cinemas",
    "Golf",
    "Ski",
    "Beach & Pool Clubs",
    "School Holidays"
  ],
  "Attractions & Tickets": [
    "Theme & water parks",
    "Attraction passes",
    "Museums",
    "Zoos & aquariums",
    "Historical sites",
    "Galleries"
  ],
  "Live Events": [
    "Concerts",
    "Theatre",
    "Live sports",
    "Special Events"
  ],
  "Indulge Yourself": [
    "Spa & massage",
    "Hot springs",
    "Wellness"
  ],
  "Lux Exclusives": [
    "The best of the best"
  ],
  "Travel Essentials": [
    "Airport lounges",
    "Luggage",
    "Airport Services",
    "Water Transfers"
  ],
  "Day Tours": [
    "Guided tours",
    "Walking tours",
    "Bike tours",
    "Hop-on-hop-off",
    "Private tours"
  ],
  "Gift Inspiration": [
    "Foodie",
    "Thrill Seeker",
    "Animal Lover",
    "Spa-goer",
    "Family",
    "Aquatic Enthusiast"
  ]
}


## Target Audiences
These are the possible target audiences. Some or all can apply (it is most common for all to apply).
- Solo
- Couple
- Group
- Family

When selecting the hero image:
- You should chose the image most appropriate to be the lead/hero image for the experience offer on our website. This is the image we show in search results, and first on the offer page.
- Use your understanding of the experience based on the offer description, and your knowledge of what customers are looking for, to guide your decision making
- The first image that Klook provided often is very edited to include a promotional overlay. We should not choose that one. (Text naturally in the image, e.g. on the side of a bus, is fine. Edited overlays are not)
- Ideally the customer would be able to look at the image and activity title and think 'I understand what that is about!'
- Use the numbered list of images provided in the prompt; pick the index that best matches the guidance.
- Return hero_image_url as the exact https URL from that list (do not respond with attachment:// references).
- If none of the supplied images are acceptable, set hero_image_index and hero_image_url to null and explain why in hero_image_reason.
"""


In [45]:
MAX_IMAGES_TO_REVIEW = 8
MODEL_NAME = "gpt-5"
REASONING_EFFORT = "medium"
MAX_OUTPUT_TOKENS = 5000
OPENAI_API_KEY_ENV = "OPENAI_API_KEY"
ENV_PRIORITIES = [
    Path(".env"),
    Path(".openai_api_key"),
]

def load_api_key_from_file(path: Path) -> str | None:
    try:
        content = path.read_text(encoding="utf-8")
    except OSError:
        return None
    for raw_line in content.splitlines():
        line = raw_line.strip()
        if not line or line.startswith("#"):
            continue
        if "=" not in line:
            continue
        key, value = line.split("=", 1)
        if key.strip() == OPENAI_API_KEY_ENV:
            return value.strip().strip('"')
    return None


def load_api_key() -> str | None:
    value = os.getenv(OPENAI_API_KEY_ENV)
    if value:
        return value.strip()
    for env_path in ENV_PRIORITIES:
        if env_path.exists():
            candidate = load_api_key_from_file(env_path)
            if candidate:
                return candidate
    return None


_client: OpenAI | None = None


def get_client() -> OpenAI:
    """Return a shared OpenAI client, raising if the API key is missing."""
    global _client
    if _client is None:
        api_key = load_api_key()
        if not api_key:
            raise RuntimeError(
                "OpenAI API key not found. Set OPENAI_API_KEY or add it to a local .env file."
            )
        _client = OpenAI()
    return _client


def summarise_packages(packages: List[Dict[str, Any]]) -> str:
    if not packages:
        return "No packages available."
    lines: List[str] = []
    for idx, package in enumerate(packages, start=1):
        name = package.get("package_name") or f"Package {idx}"
        details = package.get("sections_markdown") or "No details supplied."
        lines.append(f"Package: {name}\n{details}")
    return "\n\n".join(lines)



    
    def build_offer_prompt(offer: Dict[str, Any]) -> str:
        """Format the offer payload into a single prompt string."""
        lines = [
            f"Activity ID: {offer.get('activity_id')}",
            f"Title: {offer.get('title') or 'N/A'}",
            f"Subtitle: {offer.get('subtitle') or 'N/A'}",
            f"What we love: {offer.get('what_we_love') or 'N/A'}",
            f"Location (lat,long): {offer.get('location') or 'N/A'}",
            f"Address: {offer.get('address') or 'N/A'}",
            f"City: {offer.get('city') or 'N/A'}",
            f"Country: {offer.get('country') or 'N/A'}",
            f"Current category: {offer.get('category') or 'N/A'}",
            "",
            "Offer description markdown:",
            offer.get('description_markdown') or 'No description supplied.',
            "",
            "Packages:",
            summarise_packages(offer.get('packages') or []),
            "",
            f"Images provided (max {MAX_IMAGES_TO_REVIEW} considered):",
        ]
        image_details: List[Dict[str, Any]] = (offer.get('image_details') or [])[:MAX_IMAGES_TO_REVIEW]
        if not image_details:
            lines.append("No images available.")
        else:
            lines.append("Image metadata includes Klook's image_type to help avoid stylised banners.")
            lines.append("Always reference these numbers when returning hero_image_index and use the exact URL shown.")
            for idx, image in enumerate(image_details, start=1):
                image_type = image.get('type') or 'UNKNOWN'
                alt_text = (image.get('alt') or '').strip() or 'N/A'
                description = (image.get('description') or '').strip()
                if len(description) > 120:
                    description = f"{description[:117]}..."
                description = description or 'N/A'
                url = image.get('url') or 'N/A'
                lines.append(
                    f"[{idx}] type={image_type} alt={alt_text} desc={description} url={url}"
                )
        return "".join(lines)


def collect_response_text(response: Any) -> str:
    """Extract concatenated text from a Responses API call."""
    payload = response.to_dict() if hasattr(response, "to_dict") else response
    chunks: list[str] = []
    for item in payload.get("output", []):
        for content in item.get("content", []):
            if content.get("type") == "output_text":
                chunks.append(content.get("text", ""))
    return "".join(chunks).strip()


def parse_json_response(text: str) -> Dict[str, Any]:
    """Parse the model's JSON response, tolerating surrounding text."""
    if not text:
        return {"score": None, "reason": "Empty response from model."}
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        import re
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if match:
            try:
                return json.loads(match.group(0))
            except json.JSONDecodeError:
                pass
    return {"score": None, "reason": f"Failed to parse JSON: {text}"}



def normalise_str_list(value: Any) -> List[str]:
    """Return a list of strings for JSON list-or-string values."""
    if isinstance(value, list):
        return [str(item) for item in value if item is not None]
    if value in (None, ""):
        return []
    return [str(value)]




def grade_offer(offer: Dict[str, Any]) -> Dict[str, Any]:
    """Call the OpenAI Responses API to grade a single offer."""
    client = get_client()
    prompt = build_offer_prompt(offer)

    candidate_images: List[Dict[str, Any]] = offer.get("image_details") or []
    image_payload = [
        {"type": "input_image", "image_url": image.get("url")}
        for image in candidate_images[:MAX_IMAGES_TO_REVIEW]
        if image.get("url")
    ]
    if not image_payload:
        image_payload = [
            {"type": "input_image", "image_url": url}
            for url in (offer.get("images") or [])[:MAX_IMAGES_TO_REVIEW]
            if url
        ]

    offer_content = [{"type": "input_text", "text": prompt}] + image_payload
    response_id: str | None = None
    categories: List[str] = []
    target_audiences: List[str] = []
    hero_image_url: str | None = None
    hero_image_reason: str | None = None
    hero_image_index: int | None = None

    try:
        response = client.responses.create(
            model=MODEL_NAME,
            instructions=SYSTEM_PROMPT,
            input=[{"role": "user", "content": offer_content}],
            reasoning={"effort": REASONING_EFFORT},
            max_output_tokens=MAX_OUTPUT_TOKENS,
            metadata={
                "activity_id": str(offer.get("activity_id")),
                "activity_title": offer.get("title"),
                "activity_url": f"https://www.klook.com/en-AU/activity/{offer.get('activity_id')}",
                "activity_category": offer.get("category"),
            },
        )
        response_id = getattr(response, "id", None)
        response_text = collect_response_text(response)
        parsed = parse_json_response(response_text)
        categories = normalise_str_list(parsed.get("categories"))
        target_audiences = normalise_str_list(parsed.get("target_audiences"))

        hero_image_index_raw = parsed.get("hero_image_index")
        if isinstance(hero_image_index_raw, (int, float)):
            hero_image_index = int(hero_image_index_raw)
        elif isinstance(hero_image_index_raw, str) and hero_image_index_raw.strip():
            try:
                hero_image_index = int(hero_image_index_raw.strip())
            except ValueError:
                hero_image_index = None

        hero_image_url_raw = parsed.get("hero_image_url")
        hero_image_url = str(hero_image_url_raw).strip() if hero_image_url_raw else None
        if hero_image_url == "":
            hero_image_url = None

        if hero_image_index is not None:
            if 1 <= hero_image_index <= len(candidate_images):
                candidate_url = candidate_images[hero_image_index - 1].get("url")
                if candidate_url:
                    hero_image_url = candidate_url
            else:
                hero_image_index = None

        if hero_image_url and hero_image_url.startswith("attachment://"):
            hero_image_url = None
            if hero_image_index is not None and 1 <= hero_image_index <= len(candidate_images):
                hero_image_url = candidate_images[hero_image_index - 1].get("url")

        if hero_image_url and not hero_image_url.startswith("attachment://") and hero_image_index is None:
            for idx, image in enumerate(candidate_images, start=1):
                if image.get("url") == hero_image_url:
                    hero_image_index = idx
                    break

        if not hero_image_url and hero_image_index is None and candidate_images:
            provided_url = parsed.get("hero_image_url")
            if provided_url and isinstance(provided_url, str):
                trimmed = provided_url.strip()
                if trimmed and not trimmed.startswith("attachment://"):
                    for idx, image in enumerate(candidate_images, start=1):
                        if image.get("url") == trimmed:
                            hero_image_url = trimmed
                            hero_image_index = idx
                            break

        hero_image_reason = parsed.get("hero_image_reason")
        if isinstance(hero_image_reason, dict):
            hero_image_reason = json.dumps(hero_image_reason)
        elif hero_image_reason is None:
            hero_image_reason = ""
        else:
            hero_image_reason = str(hero_image_reason).strip()
    except Exception as exc:  # noqa: BLE001
        detail = getattr(exc, "response", None)
        extra_info = None
        if detail is not None:
            try:
                extra_info = detail.json()
            except Exception:  # noqa: BLE001
                if hasattr(detail, "text") and detail.text:
                    extra_info = detail.text
                elif hasattr(detail, "content") and detail.content:
                    extra_info = detail.content
        reason = f"Model call failed: {exc}"
        if extra_info is not None:
            reason = f"{reason} | {extra_info}"
        return {
            "activity_id": offer.get("activity_id"),
            "score": None,
            "reason": reason,
            "categories": categories,
            "target_audiences": target_audiences,
            "hero_image_index": None,
            "hero_image_url": None,
            "hero_image_reason": "",
            "response_id": None,
        }

    score = parsed.get("score")
    try:
        score_value = float(score) if score is not None else None
    except (TypeError, ValueError):
        score_value = None
    if score_value is not None:
        score_value = max(0.0, min(5.0, score_value))

    reason = parsed.get("reason") or parsed
    if isinstance(reason, dict):
        reason = json.dumps(reason)

    return {
        "activity_id": offer.get("activity_id"),
        "score": score_value,
        "reason": reason,
        "categories": categories,
        "target_audiences": target_audiences,
        "hero_image_index": hero_image_index,
        "hero_image_url": hero_image_url,
        "hero_image_reason": hero_image_reason or "",
        "response_id": response_id,
    }


def grade_offers_parallel(offers: List[Dict[str, Any]], max_workers: int | None = None) -> List[Dict[str, Any]]:
    """Grade offers in parallel using a thread pool."""
    if not offers:
        return []
    worker_count = max_workers or min(8, len(offers)) or 1
    results: List[Dict[str, Any]] = []
    with ThreadPoolExecutor(max_workers=worker_count) as executor:
        future_map = {executor.submit(grade_offer, offer): offer for offer in offers}
        for future in as_completed(future_map):
            offer = future_map[future]
            try:
                result = future.result()
            except Exception as exc:  # noqa: BLE001
                result = {
                    "activity_id": offer.get("activity_id"),
                    "score": None,
                    "reason": f"Unexpected error: {exc}",
                    "categories": [],
                    "target_audiences": [],
                    "response_id": None,
                }
            results.append(result)
    results.sort(key=lambda item: (item.get("activity_id"), item.get("reason")))
    return results


In [46]:
raw_offers = load_offers(OFFERS_DIR)
structured_offers = [structure_activity(item["activity"], item["path"]) for item in raw_offers]
print(f"Loaded {len(structured_offers)} offers from {OFFERS_DIR.resolve()}")
overview_records = [
    {
        "activity_id": offer.get("activity_id"),
        "title": offer.get("title"),
        "city": offer.get("city"),
        "country": offer.get("country"),
        "category": offer.get("category"),
        "status": offer.get("status"),
        "image_count": len(offer.get("images") or []),
        "package_count": len(offer.get("packages") or []),
    }
    for offer in structured_offers
]
overview_df = pd.DataFrame(overview_records)
overview_df


Loaded 5 offers from /Users/williamritossa/Documents/klook-offer-curation/offers


,activity_id,title,city,country,category,status,image_count,package_count
0,107217,Go City - New York Pass,New York,United States,Attractions,None,9,8
1,1592,Go City Las Vegas Explorer Pass,Las Vegas,United States,Attractions,None,10,4
2,18333,Chicago CityPASS®,Chicago,United States,Attractions,None,9,1
3,34300,SEA LIFE Aquarium Ticket in Orlando,Orlando,United States,Attractions,None,5,1
4,6227,Go City - Miami All-Inclusive Pass,Miami,United States,Attractions,None,12,4


In [47]:
if structured_offers:
    sample_offer = structured_offers[0]
    sample_summary = {
        "activity_id": sample_offer.get("activity_id"),
        "title": sample_offer.get("title"),
        "subtitle": sample_offer.get("subtitle"),
        "what_we_love": sample_offer.get("what_we_love"),
        "location": sample_offer.get("location"),
        "category": sample_offer.get("category"),
        "images": sample_offer.get("images"),
        "description_markdown": sample_offer.get("description_markdown"),
    }
    pd.Series(sample_summary)
else:
    print("No offers found.")


In [48]:
if structured_offers:
    sample_packages = pd.DataFrame(structured_offers[0].get("packages") or [])
    sample_packages


In [49]:
offers_to_grade = [
    offer for offer in structured_offers
    if (offer.get("status") or "").upper() != "CURATED"
]
print(f"Queued {len(offers_to_grade)} offers for grading.")


Queued 5 offers for grading.


In [50]:

if offers_to_grade:
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("OPENAI_API_KEY is not set in this kernel session. Skipping grading.")
    else:
        grading_results = grade_offers_parallel(offers_to_grade)
        results_df = pd.DataFrame(grading_results)
        if "hero_image_index" not in results_df:
            results_df["hero_image_index"] = None
        if "hero_image_url" not in results_df:
            results_df["hero_image_url"] = None
        if "hero_image_reason" not in results_df:
            results_df["hero_image_reason"] = ""
        results_df["activity_url"] = results_df["activity_id"].apply(
            lambda x: f"https://www.klook.com/en-AU/activity/{x}" if pd.notna(x) else None
        )
        results_df["log_url"] = results_df["response_id"].apply(
            lambda resp: f"https://platform.openai.com/logs/{resp}"
            if isinstance(resp, str) and resp
            else None
        )
        results_df = results_df[
            [
                "activity_id",
                "activity_url",
                "hero_image_index",
                "hero_image_url",
                "hero_image_reason",
                "categories",
                "target_audiences",
                "score",
                "reason",
                "log_url",
            ]
        ]
        display(results_df)
else:
    print("No offers require grading.")


,activity_id,activity_url,hero_image_index,hero_image_url,hero_image_reason,categories,target_audiences,score,reason,log_url
0,1592,https://www.klook.com/en-AU/activity/1592,7,https://res.klook.com/image/upload/activities/iey1ig0svcj8ekud5p9i.jpg,"Select image 7 (Madame Tussauds) as the most evocative, high-quality representation of a popular attraction on the pass without promotional overlays; however, no direct image URL was provided in the list.",[Attraction passes],"[Solo, Couple, Group, Family]",4.0,"Strong, clear product (multi-attraction city pass) suited to our audience. Title is clear; image set mostly relevant (avoid #1 due to promotional overlay and #3 due to firearms display). Current category should be refined to Attractions & Tickets > Attraction passes. Description is informative but contains a key inconsistency: validity is stated as 60 days in multiple places, then 30 days after activation later—this needs correction. Location (Las Vegas) is correct. Overall suitable with minor copy edits and image selection refinement.",https://platform.openai.com/logs/resp_0c833f6968e0ec680068db626428f881a2a21bb8f8c7d821f3
1,6227,https://www.klook.com/en-AU/activity/6227,4,https://res.klook.com/image/upload/activities/xv9yytshdplyl622s5lr.jpg,"Image 4 (Big Bus Miami with skyline) clearly communicates a city sightseeing pass, is high quality, location-relevant and free of promotional overlays. Note: a direct URL for the supplied images was not provided in the brief; if available, use image #4’s URL.",[Attraction passes],"[Solo, Couple, Group, Family]",4.0,"Strong, clear title and comprehensive description for a multi-day all-inclusive city pass covering 30+ Miami-area attractions. Images are relevant; avoid image #1 due to heavy promotional overlay. Best category is Attractions & Tickets > Attraction passes (current category simply ‘Attractions’ is too broad). Location is set to a specific venue (Bayfront Park amphitheater) though the product is app-based and citywide—adjust to citywide location to prevent confusion. Suitable for all audience types.",https://platform.openai.com/logs/resp_0c9df6f23ab8da4d0068db6263f2c0819fab319a830e3cb96c
2,18333,https://www.klook.com/en-AU/activity/18333,3,https://res.klook.com/image/upload/activities/u5rxjai8dibz7z6hpb2a.jpg,"Image 3 (Skydeck Chicago glass ledge) is an iconic, high-quality shot that instantly communicates a top included attraction and the panoramic city views the pass unlocks. It’s clean with no promotional overlay. Note: the prompt doesn’t provide direct https URLs for the images, so the URL cannot be populated.",[Attraction passes],"[Solo, Couple, Group, Family]",5.0,"Strong, clear title and accurate positioning as a multi-attraction pass; comprehensive inclusions, terms and addresses; correct city location; images are relevant (avoid Image 1 due to heavy promotional overlay). Category should be Attractions & Tickets > Attraction passes. Suitable for all traveler types.",https://platform.openai.com/logs/resp_0c00986c4ebae92b0068db62642b6c81919630af41d69a316c
3,34300,https://www.klook.com/en-AU/activity/34300,1,https://res.klook.com/image/upload/activities/aldherl7tqtxgym8ilhd.jpg,"Image 1 best represents the experience by showcasing the signature 360-degree ocean tunnel with marine life, instantly communicating 'aquarium visit' with no promotional overlays. Note: the provided images lack accessible https URLs in the listing, so a direct link cannot be supplied.",[Zoos & aquariums],"[Solo, Couple, Group, Family]",4.0,"Strong, clear title and accurate positioning as an aquarium ticket. Imagery is relevant and family-friendly; Image 1 is an ideal hero. Category should be Attractions & Tickets > Zoos & aquariums. Description covers highlights, inclusions, hours and access clearly. Location coordinates align with ICON Park area; minor red flag: address line has a typo ('Orando' instead of 'Orlando'). Overall suitable for LE with small copy edit.",https://platform.openai.com/log

## Export to CSV

In [51]:
if 'results_df' in locals():
    export_path = Path('graded_offers.csv')
    results_df.to_csv(export_path, index=False)
    print(f'Exported results to {export_path.resolve()}')
else:
    print('results_df is not defined. Run the grading cell first.')


Exported results to /Users/williamritossa/Documents/klook-offer-curation/graded_offers.csv
